# Cobalamin metabolism in PS00212, iTps1432, and iML1515

In [121]:
import cobra


def open_inorganic_exchanges(
        model: cobra.Model,
        lower_bound: float = -1000,
        upper_bound: float = 1000,
        include: list = None,
        inplace: bool = True) -> cobra.Model:
    """
    Open all inorganic exchanges in a model and close organic exchanges,
    except for indicated ones.

    Parameters
    ----------
    model : cobra.Model
        Model to open exchanges in.
    include: list, optional
        List of exchange IDs to be opened besides inorganic ones. Default is None.
    inplace : bool, optional
        If True, open exchanges in place. If False, return a copy of the model with opened
        exchanges. Default is True.

    Returns
    -------
    cobra.Model
        Model with opened exchanges.
    """
    if inplace:
        model = model
    else:
        model = model.copy()

    for rxn in model.exchanges:
        met = [met for met in rxn.metabolites][0]
        if (met.formula is not None) and ("C" not in met.formula):
            rxn.lower_bound = lower_bound
            rxn.upper_bound = upper_bound
        else:
            rxn.lower_bound = 0

    return model


iML1515 = cobra.io.read_sbml_model("iML1515.xml")
ps00212 =  cobra.io.read_sbml_model("gems/MAG_00212_pseudonitzschia_photoeuk.xml")
iTPS1432 = cobra.io.read_sbml_model("iTps1432_high.xml")

# cobalamin
methylcbl_00212 = ps00212.metabolites.get_by_id("C06453_c")
cbl_00212 = ps00212.metabolites.get_by_id("cbl1_c")

# cobalamin-dependent methionine synthase
meth_00212_a = ps00212.reactions.get_by_id("RXN-21539")
meth_00212_b = ps00212.reactions.get_by_id("HOMOCYSMETB12-RXN")

meth_itps_a = iTPS1432.reactions.get_by_id("METS1_B12_c")
meth_itps_b = iTPS1432.reactions.get_by_id("METS2_B12_c")

# cobalamin-independent methionine synthase (iML1515)
mete_iml1515 = iML1515.reactions.get_by_id("METS")

## Methionine synthase

It looks like ps00212 is fails at capturing the importance of MetH. Since growth is not affected by bocking it. Methionine is synthesized by a number of other reactions. However, in iML1515, MetE is still required for growth. Why this doesn't happen in ps00212 (MetH) is unclear. Perhaps due to a leak in tetrahydrofolate?

### iML1515

In [8]:
mete_iml1515

Reaction identifier,METS
Name,Methionine synthase
Memory address,0x7f9f39358b50
Stoichiometry,"5mthf_c + hcys__L_c --> h_c + met__L_c + thf_c 5-Methyltetrahydrofolate + L-Homocysteine --> H+ + L-Methionine + 5,6,7,8-Tetrahydrofolate"
GPR,b4019 or b3829
Lower bound,0.0
Upper bound,0


In [53]:
mete_iml1515.upper_bound = 1000
iML1515.optimize()

,fluxes,reduced_costs
CYTDK2,0.00000,-6.938894e-18
XPPT,0.00000,-7.523353e-03
HXPRT,0.00000,-7.523353e-03
NDPK5,0.00000,-0.000000e+00
SHK3Dr,0.33424,0.000000e+00
...,...,...
MPTS,0.00000,-2.775558e-17
MOCOS,0.00000,0.000000e+00
BMOGDS2,0.00000,0.000000e+00
FESD2s,0.00000,0.000000e+00


What other reactions produce met__L_c in iML1515?

In [54]:
[
    (rxn.id, rxn.name, rxn.lower_bound, rxn.upper_bound, rxn.flux)
    for rxn in iML1515.reactions
    if ("met__L_c" in [met.id for met in rxn.products])
    ]

[('HCYSMT', 'Homocysteine S-methyltransferase', 0.0, 1000.0, 0.0),
 ('METS', 'Methionine synthase', 0.0, 1000, 0.13537241302173342),
 ('METSOXR1', 'L-methionine-S-oxide reductase', 0.0, 1000.0, 0.0),
 ('METSOXR2', 'L-methionine-R-sulfoxide reductase', 0.0, 1000.0, 0.0),
 ('METabcpp',
  'L-methionine transport via ABC system (periplasm)',
  0.0,
  1000.0,
  0.0),
 ('HCYSMT2', 'Homocysteine Methyltransferase', 0.0, 1000.0, 0.0),
 ('CPPPGO2',
  'Oxygen Independent coproporphyrinogen-III oxidase',
  0.0,
  1000.0,
  0.0),
 ('BTS5', 'Biotin synthase', 0.0, 1000.0, 1.7539944288539496e-06),
 ('LIPOS', 'Lipoate synthase', 0.0, 1000.0, 0.0),
 ('TYRL', 'Tyrosine lyase', 0.0, 1000.0, 0.0001955703788172154)]

### ps00212

In [4]:
meth_00212_a

Reaction identifier,RXN-21539
Name,cobalamin-dependent methionine synthase
Memory address,0x7f9f366b43d0
Stoichiometry,5-METHYL-THF-GLU-N_c + cbl1_c --> C06453_c + THF-GLU-N_c a 5-methyltetrahydrofolate + cbl1 --> Methylcobalamin + a tetrahydrofolate
GPR,TARA_ARC_108_MAG_00212_000000004337_12_1
Lower bound,0.0
Upper bound,0


In [5]:
meth_00212_b

Reaction identifier,HOMOCYSMETB12-RXN
Name,methionine synthase
Memory address,0x7f9f36796a10
Stoichiometry,C06453_c + hcys__L_c --> cbl1_c + 4.0 h_c + met__L_c Methylcobalamin + L-Homocysteine --> cbl1 + 4.0 H+ + L-Methionine
GPR,TARA_ARC_108_MAG_00212_000000004337_12_1
Lower bound,0.0
Upper bound,0


In [111]:
# MetH
meth_00212_a.upper_bound = 1000
meth_00212_b.upper_bound = 1000
ps00212.reactions.get_by_id('EX_met__L_e').lower_bound = 0

ps00212.optimize()

,fluxes,reduced_costs
ATPCS_c,0.000000e+00,0.0
CISY_m,0.000000e+00,0.0
PDH_E1a_h,0.000000e+00,0.0
ADH_c,0.000000e+00,0.0
GAPDH_h,0.000000e+00,0.0
...,...,...
PREPHENATEDEHYDRAT-RXN,0.000000e+00,0.0
EX_cbl1_e,-3.422730e-07,-0.0
EX_adocbl_e,0.000000e+00,-0.0
cbl1_transport,3.422730e-07,0.0


In [61]:
[
    (rxn.id, rxn.name, rxn.lower_bound, rxn.upper_bound, rxn.flux)
    for rxn in ps00212.reactions
    if ("met__L_c" in [met.id for met in rxn.products])
    ]

[('AMPMS_h',
  '4-amino-2-methyl-5-phosphomethylpyrimidine synthetase',
  0.0,
  1000.0,
  0.0),
 ('BTS_m', 'Biotin synthase', 0.0, 1000.0, 0.0),
 ('UNK3', '2-keto-4-methylthiobutyrate transamination', -1000.0, 1000.0, 0.0),
 ('BHMT',
  'Betaine-homocysteine S-methyltransferase',
  0.0,
  1000.0,
  3.9757413773623007),
 ('CPPPGO2', 'CPPPGO2', 0.0, 1000.0, 0.0),
 ('HOMOCYSMET-RXN',
  '5-methyltetrahydropteroyltriglutamate-homocysteine S-methyltransferase',
  -1000.0,
  1000.0,
  0.0),
 ('HOMOCYSMETB12-RXN', 'methionine synthase', 0.0, 1000, 0.0),
 ('RXN0-5063', 'isopentenyl-adenosine tRNA methylthiolase', 0.0, 1000.0, 0.0),
 ('METt2r', 'METt2r', 0.0, 1000.0, 0.0),
 ('ORNITHINE--OXO-ACID-AMINOTRANSFERASE-RXN-L-ORNITHINE/CPD-479//MET/L-GLUTAMATE_GAMMA-SEMIALDEHYDE.56.',
  'ORNITHINE--OXO-ACID-AMINOTRANSFERASE-RXN-L-ORNITHINE/CPD-479//MET/L-GLUTAMATE_GAMMA-SEMIALDEHYDE.56.',
  -1000.0,
  1000.0,
  6.917100408379479),
 ('TRANS-RXN18ZY-5-MET/NA+//MET/NA+.17.',
  'TRANS-RXN18ZY-5-MET/NA+//MET

Alright, so these two reactions are producing all met__L_c:

* ORNITHINE--OXO-ACID-AMINOTRANSFERASE-RXN-L-ORNITHINE/CPD-479//MET/L-GLUTAMATE_GAMMA-SEMIALDEHYDE.56.
* TRANS-RXN18ZY-5-MET/NA+//MET/NA+.17. (Import met__L_c)

When import blocked:
* BHMT


In [58]:
ps00212.reactions.get_by_id("ORNITHINE--OXO-ACID-AMINOTRANSFERASE-RXN-L-ORNITHINE/CPD-479//MET/L-GLUTAMATE_GAMMA-SEMIALDEHYDE.56.")

Reaction identifier,ORNITHINE--OXO-ACID-AMINOTRANSFERASE-RXN-L-ORNITHINE/CPD-479//MET/L-GLUTAMATE_GAMMA-SEMIALDEHYDE.56.
Name,ORNITHINE--OXO-ACID-AMINOTRANSFERASE-RXN-L-ORNITHINE/CPD-479//MET/L-GLUTAMATE_GAMMA-SEMIALDEHYDE.56.
Memory address,0x7f02270bd3d0
Stoichiometry,2kmb_c + orn_c <=> glu5sa_c + met__L_c 4-Methylthio-2-oxobutanoic acid + Ornithine <=> L-Glutamate 5-semialdehyde + L-Methionine
GPR,TARA_ARC_108_MAG_00212_000000002402_1_2 or TARA_ARC_108_MAG_00212_000000004614_7_2
Lower bound,-1000.0
Upper bound,1000.0


In [59]:
ps00212.reactions.get_by_id("TRANS-RXN18ZY-5-MET/NA+//MET/NA+.17.")

Reaction identifier,TRANS-RXN18ZY-5-MET/NA+//MET/NA+.17.
Name,TRANS-RXN18ZY-5-MET/NA+//MET/NA+.17.
Memory address,0x7f021ef806d0
Stoichiometry,met__L_e + na1_e <=> met__L_c + na1_c met__L + Sodium cation <=> L-Methionine + Sodium cation
GPR,TARA_ARC_108_MAG_00212_000000002095_4_1
Lower bound,-10.0
Upper bound,1000.0


In [62]:
ps00212.reactions.get_by_id("BHMT")

Reaction identifier,BHMT
Name,Betaine-homocysteine S-methyltransferase
Memory address,0x7f0225b0a090
Stoichiometry,glyb_c + hcys__L_c --> dmgly_c + met__L_c Betaine + L-Homocysteine --> Dimethylglycine + L-Methionine
GPR,TARA_ARC_108_MAG_00212_000000000556_8_1
Lower bound,0.0
Upper bound,1000.0


### iTps1432

In [77]:
# for rxn in iTPS1432.exchanges:
#     # if "photon" in rxn.id:
#     rxn.lower_bound = -10000

In [88]:
# Open inorganic exchanges
iTPS1432 = open_inorganic_exchanges(iTPS1432, lower_bound=-10000, upper_bound=10000)

In [94]:
meth_itps_a.upper_bound = 1 * 10000
meth_itps_b.upper_bound = 1 * 10000

iTPS1432.optimize()

,fluxes,reduced_costs
IDP_m,25.681116,0.000000e+00
ATAM_c,855.417209,0.000000e+00
GLTS_c,42.438215,1.110223e-16
GLNA_h,0.000000,1.110223e-16
GLNA_m,0.000000,1.110223e-16
...,...,...
biomass_chitin_c,0.731840,0.000000e+00
biomass_frustule_c,0.760210,0.000000e+00
bof200_c,18.954214,1.859624e-15
biomass_EPS_c,0.000000,-7.000281e+00


Interestingly, iTps1432 is sensitive to blocking MetH... why?? What are the other reactions producing met__L_c?

In [95]:
[
    (rxn.id, rxn.name, rxn.lower_bound, rxn.upper_bound, rxn.flux)
    for rxn in iTPS1432.reactions
    if ("met__L_c" in [met.id for met in rxn.products])
    ]

[('MTOBTA_c',
  '4-Methylthio-2-oxobutanoate transaminase',
  -10000.0,
  10000.0,
  -1.2288120631889707),
 ('METt_m',
  'Amino acid transporter (Met-L), mitochondrial',
  -10000.0,
  10000.0,
  0.0),
 ('METt_h',
  'L-methionine transport from chloroplast to the cytosol',
  -10000.0,
  10000.0,
  0.0),
 ('BHMT_c', 'Betaine-homocysteine S-methyltransferase', 0.0, 10000.0, 0.0),
 ('METS2_B12_c',
  'Methionine synthase (B12 dependent) part 2',
  0.0,
  10000,
  11.506067679203841)]

## Cobalamin exchanges

Alright, so I added the modified vitamin biomass pseudoreaction and now ps00212 is auxotroph for adocbl (or cbl1). However, the model still doesn't care about blocking MetH...

Alright, now is sensitive to blocking MetH, after opening only inorganic exchanges. However, I see that it is realising NH4+ but not importing it? or any other source of N besides vitamins. Pseudo-nitzschia is not a diazotroph, so it should be importing NH4+. There must be some mass inbalance in the model.

In [128]:
ps00212 = open_inorganic_exchanges(ps00212, lower_bound=-1000)

In [124]:
adocbl_ex = ps00212.reactions.get_by_id("EX_adocbl_e")
cbl1_ex = ps00212.reactions.get_by_id("EX_cbl1_e")
cbl1_transport = ps00212.reactions.get_by_id("cbl1_transport")
cbl_adocbl_inter = ps00212.reactions.get_by_id("cbl_interconversion_e")

# biotin
bio_ex = ps00212.reactions.get_by_id("EX_biotin_e")

In [125]:
ps00212.reactions.get_by_id("EX_photon_e").upper_bound = 0
ps00212.reactions.get_by_id("EX_photon_e").lower_bound = -1000
ps00212.reactions.get_by_id("EX_photon_e")

Reaction identifier,EX_photon_e
Name,Photon exchange
Memory address,0x7fa3ae1bbd50
Stoichiometry,photon_e <-- Photon <--
GPR,
Lower bound,-1000
Upper bound,0


In [129]:
# Pseudo-nitzschia
# cobalamin uptake
cbl1_ex.lower_bound = 0
adocbl_ex.lower_bound = -1000
cbl1_transport.lower_bound, cbl1_transport.upper_bound = -1000, 1000
cbl_adocbl_inter.lower_bound, cbl_adocbl_inter.upper_bound = -1000, 1000

bio_ex.lower_bound = -1000



sol = ps00212.optimize()
sol

,fluxes,reduced_costs
ATPCS_c,0.000000e+00,0.0
CISY_m,0.000000e+00,0.0
PDH_E1a_h,0.000000e+00,0.0
ADH_c,0.000000e+00,0.0
GAPDH_h,1.480909e-01,0.0
...,...,...
PREPHENATEDEHYDRAT-RXN,2.835205e-04,0.0
EX_cbl1_e,0.000000e+00,-0.0
EX_adocbl_e,0.000000e+00,-0.0
cbl1_transport,7.871807e-12,0.0


In [130]:
sol_ex = sol[[rxn.id for rxn in ps00212.exchanges]]
sol_ex = sol_ex[sol_ex != 0]
sol_ex = sol_ex.sort_values(ascending=True)
sol_ex.to_csv('ps00212_exchanges.csv')

## Check mass inbalances in ps00212

* Starting with NH4+ as it should not export it

In [93]:
for rxn in ps00212.reactions:
    imbalances = rxn.check_mass_balance()
    if len(imbalances) > 0:
        if "N" in imbalances:
            print(" ")
            print(rxn.id, rxn.name, rxn.reaction, imbalances)
            for met in rxn.metabolites:
                print(" ", met.id, met.name, met.formula, met.charge)


Some weird imbalanced reactions that could be producing N out of nothing, such as DM_dgta205n31619Z_c, or SK_Asn_X_Ser_Thr_c.

Alrigth, what's happening is that there are a bunch of "sink" reactions which have incorrect bounds, and are in fact producing N out of nothing. Let's fix that.

I think there was a version of this model without sink reactions (no SK)

In [89]:
ps00212.reactions.get_by_id("SK_Asn_X_Ser_Thr_c")

Reaction identifier,SK_Asn_X_Ser_Thr_c
Name,Sink Asn-X-Ser Thr c
Memory address,0x7fa3b46ee890
Stoichiometry,Asn_X_Ser_Thr_c <-- Protein-linked asparagine residue (N-glycosylation site) <--
GPR,
Lower bound,-0.5
Upper bound,0.0


In [92]:
ps00212.reactions.get_by_id("DM_dgta205n31619Z_c")

Reaction identifier,DM_dgta205n31619Z_c
Name,R_DM_dgta205n31619Z_c
Memory address,0x7fa3b43eec10
Stoichiometry,"dgta205n31619Z_c --> Diacylglyceryl-N,N,N-trimethyl-beta-alanine (20:5(5Z,8Z,11Z,14Z,17Z)/16:1(9Z)) -->"
GPR,
Lower bound,0.0
Upper bound,1000.0


Let's block sink reactions

In [133]:
for rxn in ps00212.reactions:
    if "SK_" in rxn.id:
        rxn.lower_bound = -1000
        rxn.upper_bound = 1000


sol = ps00212.optimize()
sol

,fluxes,reduced_costs
ATPCS_c,1.268258e-03,0.0
CISY_m,0.000000e+00,0.0
PDH_E1a_h,0.000000e+00,0.0
ADH_c,0.000000e+00,0.0
GAPDH_h,3.823396e-02,0.0
...,...,...
PREPHENATEDEHYDRAT-RXN,2.835205e-04,0.0
EX_cbl1_e,0.000000e+00,-0.0
EX_adocbl_e,0.000000e+00,-0.0
cbl1_transport,7.871807e-12,0.0


In [134]:
sol_ex = sol[[rxn.id for rxn in ps00212.exchanges]]
sol_ex = sol_ex[sol_ex != 0]
sol_ex = sol_ex.sort_values(ascending=True)
sol_ex.to_csv('ps00212_exchanges.csv')

## Investigating THF metabolism in Pseudo-nitzschia


Big difference:

* THF included in iML1515's biomass reaction but absent from PS biomass reaction!

In [6]:
thf_00212 = ps00212.metabolites.get_by_id("THF-GLU-N_c")
thf_00212

Metabolite identifier,THF-GLU-N_c
Name,a tetrahydrofolate
Memory address,0x7f9f397a2ad0
Formula,C19H21N7O5
Compartment,c
In 7 reaction(s),"GLYOHMETRANS-RXN, RXN-21539, GCVT-RXN, DIHYDROFOLATEREDUCT-RXN, FORMATETHFLIG-RXN, 3-CH3-2-OXOBUTANOATE-OH-CH3-XFER-RXN, AICARTRANSFORM-RXN"


In [9]:
thf_iml1515 = iML1515.metabolites.get_by_id('thf_c')
thf_iml1515

Metabolite identifier,thf_c
Name,"5,6,7,8-Tetrahydrofolate"
Memory address,0x7f9f3a5017d0
Formula,C19H21N7O6
Compartment,c
In 12 reaction(s),"GARFT, BIOMASS_Ec_iML1515_WT_75p37M, FTHFD, FTHFLi, ULA4NFT, AICART, GHMT2r, METS, MOHMT, BIOMASS_Ec_iML1515_core_75p37M, GLYCL, DHFR"


## Biomass reaction in Pseudo-nitzschia

* Default: bof_c: biomass equation for model testing

* M_BIOMASS_c (metabolite)

In [14]:
ps00212.objective.variables

{0 <= bof_c <= 1000.0, 0 <= bof_c_reverse_660d5 <= -0.0}

## Investigating iTps1432 of Thalasospira

This model has a "vitamin" component in its biomass reaction (R_biomass_vit_c) which includes adocbl. We could add this component to our bof in ps00212, or just take iTps1432's definition of biomass (since it is manually curated)

## Adapt vitamin biomass component to Ps00212

<reaction id="R_biomass_vit_c" name="Biomass component: Vitamins" reversible="false" fast="false" fbc:lowerFluxBound="cobra_0_bound" fbc:upperFluxBound="cobra_default_ub">
<notes>
    <html xmlns="http://www.w3.org/1999/xhtml">
    <p>SUBSYSTEM: Modeling: Biomass reaction</p>
    </html>
</notes>
<listOfReactants>
    <speciesReference species="M_btn_c" stoichiometry="0.00180467621671466" constant="true"/>
    <speciesReference species="M_pnto__R_c" stoichiometry="0.0312336793120744" constant="true"/>
    <speciesReference species="M_thm_c" stoichiometry="0.158396573308146" constant="true"/>
    <speciesReference species="M_pydxn_c" stoichiometry="0.011841652061051" constant="true"/>
    <speciesReference species="M_adocbl_c" stoichiometry="4.72255301000285e-08" constant="true"/>
    <speciesReference species="M_ribflv_c" stoichiometry="0.0764535255651577" constant="true"/>
    <speciesReference species="M_fol_c" stoichiometry="0.0573780622072159" constant="true"/>
    <speciesReference species="M_ascb__L_c" stoichiometry="3.25323755030631" constant="true"/>
    <speciesReference species="M_amet_c" stoichiometry="0.000163843675857242" constant="true"/>
    <speciesReference species="M_avite1_c" stoichiometry="0.71543826528432" constant="true"/>
</listOfReactants>
<listOfProducts>
    <speciesReference species="M_biomass_vit_c" stoichiometry="1" constant="true"/>
</listOfProducts>
</reaction>

In [16]:
itps1432 = cobra.io.read_sbml_model('/home/robaina/Documents/NewAtlantis/iTps1432_high.xml')
itps1432

Name,iTps1432
Memory address,7f9f37d733d0
Number of metabolites,2792
Number of reactions,6079
Number of genes,1432
Number of groups,93
Objective expression,4.0*DM_biomass_c - 4.0*DM_biomass_c_reverse_c49c1
Compartments,"plastid, mitochondria, cytosol, peroxisome, thylakoid membrane, extracellular"
